## Setup

In [3]:
from openai import OpenAI
import backoff
from dotenv import load_dotenv
import os
from fastapi import HTTPException

import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
user_message = "What is the position of the CDU on environmental policy?"
thread = "thread_Co1ZF7iG67Ixtoh7siBZ9w17"
assistant = "asst_hAFjnviViYmwcv6CiO07DRgh"

In [5]:
# Create a message to append to our thread
bot_message = client.beta.threads.messages.create(
    thread_id=thread, role='user', content=user_message)
# Execute our run
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread,
    assistant_id=assistant,
)
response = client.beta.threads.messages.list(thread_id=thread, run_id=run.id)
response

SyncCursorPage[Message](data=[Message(id='msg_zpEQ914ioFWBPfZFiCFHKPhd', assistant_id='asst_hAFjnviViYmwcv6CiO07DRgh', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=1437, file_citation=FileCitation(file_id='file-pXAKBOR2c84Z9ezkSYXj2ws9', quote=None), start_index=1425, text='【6:0†Quelle】', type='file_citation')], value='Vielen Dank für Ihre Frage zur Umweltpolitik der CDU. Die CDU setzt sich für eine "vertretbare und durchdachte Umweltpolitik" ein, die globale Zusammenarbeit betont und versucht, eine Balance zwischen ökologischer Verantwortung und wirtschaftlicher Entwicklung zu finden. Hier sind einige der wichtigsten Punkte ihrer Umweltpolitik:\n\n1. **Nachhaltige Energieversorgung:** Förderung erneuerbarer Energien wie Windkraft und Solarenergie, um die Abhängigkeit von fossilen Brennstoffen zu reduzieren und eine umweltfreundliche Energieversorgung sicherzustellen.\n\n2. **Energieeffizienz und Innovation:** Verb

In [25]:
response.data[0].content[0].text.annotations[0].file_citation.file_id

'file-pXAKBOR2c84Z9ezkSYXj2ws9'

In [6]:
import re
from typing import List

def extract_citations(response) -> List[str]:
    """
    Extracts the citations from the response content.
    
    Parameters:
    response (SyncCursorPage[Message]): The response object containing messages.
    
    Returns:
    List[str]: A list of citations found in the message content.
    """
    citations = []
    for message in response.data:
        for content_block in message.content:
            if isinstance(content_block, TextContentBlock):
                text_content = content_block.text.value
                citations.extend(re.findall(r'【\d+:\d+†Quelle】', text_content))
    return citations

In [7]:
citations = extract_citations(response)
citations

NameError: name 'TextContentBlock' is not defined

In [43]:
client.beta.vector_stores.file_batches.list_files(vector_store_id='vs_irHI3SzaGFUo8Hu20DFqThpc', batch_id='vsfb_fd215f6f863f4a86965be6e206d35454')

SyncCursorPage[VectorStoreFile](data=[VectorStoreFile(id='file-JJwGAvONFPfg5uta5MYKdgiw', created_at=1716808241, last_error=None, object='vector_store.file', status='completed', usage_bytes=None, vector_store_id='vs_irHI3SzaGFUo8Hu20DFqThpc'), VectorStoreFile(id='file-7BS9opE9sX7JzjfrFR9B90bc', created_at=1716808241, last_error=None, object='vector_store.file', status='completed', usage_bytes=None, vector_store_id='vs_irHI3SzaGFUo8Hu20DFqThpc'), VectorStoreFile(id='file-bssLEKFoeKt8eujiImgfzCaj', created_at=1716808241, last_error=None, object='vector_store.file', status='completed', usage_bytes=None, vector_store_id='vs_irHI3SzaGFUo8Hu20DFqThpc'), VectorStoreFile(id='file-LNGmxxL5LGiKfIApiKVYSJJ5', created_at=1716808241, last_error=None, object='vector_store.file', status='completed', usage_bytes=None, vector_store_id='vs_irHI3SzaGFUo8Hu20DFqThpc'), VectorStoreFile(id='file-apG4Nas7BVCgFnDBdnXme3In', created_at=1716808241, last_error=None, object='vector_store.file', status='completed'

In [ ]:
client.beta.vector_stores.files.upload_and_poll(vector_store_id='vs_irHI3SzaGFUo8Hu20DFqThpc', batch_id='vsfb_fd215f6f863f4a86965be6e206d35454', file='vsf_

FileNotFoundError: [Errno 2] No such file or directory: 'data/ABG.pdf'

## Checkers

In [34]:
response = client.beta.assistants.list()
print(response)

SyncCursorPage[Assistant](data=[Assistant(id='asst_TleKd7WyfFmMKCCggzlbwRLd', created_at=1716803821, description=None, instructions="You're an AI designed to engage in casual conversation with your discussion partner. Your goal is to have a light-hearted \n                            and fun conversation with your partner. Make sure to be friendly and introduce yourself. Ask about their day and interests. \n                            Share some fun facts or jokes and ask about their opinion on various topics. Make sure to discuss in german and make sure \n                            not to overwhelm your discussion partner with too many questions or too much information. Limit yourself to 1 question per message.\n                            ", metadata={}, model='gpt-4o', name='Casual Assistant', object='assistant', tools=[FileSearchTool(type='file_search')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearc

In [35]:
client.beta.assistants.delete(response.data[0].id)

AssistantDeleted(id='asst_TleKd7WyfFmMKCCggzlbwRLd', deleted=True, object='assistant.deleted')

In [40]:
response = client.beta.vector_stores.list()
print(response.data)

[VectorStore(id='vs_irHI3SzaGFUo8Hu20DFqThpc', created_at=1716808221, file_counts=FileCounts(cancelled=0, completed=35, failed=0, in_progress=0, total=35), last_active_at=1716808255, metadata={}, name='Party Programs', object='vector_store', status='completed', usage_bytes=6851336, expires_after=None, expires_at=None)]


In [37]:
client.beta.vector_stores.delete(response.data[0].id)

VectorStoreDeleted(id='vs_1oaow1PUlLGDgR9XG4uFL5Ul', deleted=True, object='vector_store.deleted')

In [23]:
response.data[0].id

'asst_TwWxtHZvL7Yyt6zEWV1wZ1k5'

In [36]:
# Helper function to create a question conversation
def get_question_conversation():
    """
    Returns a conversation starter for a question discussion.
    """
        
    system_prompt = f"""
    You're an AI designed to check the quality of the answers provided by another assistant.
    Your only job is to check that there is only one question in every message. 
    If there is more than one question, you should rewrite the message to contain only one question.
    Pick the most critical question in the message and keep it.
    Do not change the rest of the message unless you have to in order to adjust for the context of the question.
    """

    conversation_start = [{
            "role": "user",
            "content": system_prompt,
        }]
    return conversation_start

In [37]:
# Helper Function to create a new question conversation thread
@backoff.on_exception(backoff.expo, Exception, max_tries=5)
async def create_question_thread(assistant):
    """
    Takes assistant as input.
    
    Creates a new question conversation thread.
    
    Returns the created thread.
    """
    
    try:
        # Create a new conversation thread
        thread = client.beta.threads.create(
                            messages=get_question_conversation(),
                            )
        
        # Create and poll the run
        run = client.beta.threads.runs.create_and_poll(
            thread_id=thread.id, assistant_id=assistant
        )
        client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id)
        return thread
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Failed to create question conversation: {str(e)}")


In [38]:
thread = create_question_thread("asst_pluDj7W9S65tz0yHR5w0t7nm")

In [41]:
thread = client.beta.threads.create(
                messages=get_question_conversation(),
                )

In [43]:
run = client.beta.threads.runs.create_and_poll(
            thread_id=thread.id, assistant_id="asst_pluDj7W9S65tz0yHR5w0t7nm"
        )

In [45]:
run = client.beta.threads.runs.create_and_poll(
            thread_id=thread.id, assistant_id=assistant
        )

SyncCursorPage[Message](data=[Message(id='msg_hHncd621QmjDxbsQEiRAvQUw', assistant_id='asst_pluDj7W9S65tz0yHR5w0t7nm', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Understood, please provide the messages you need reviewed.'), type='text')], created_at=1716802423, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_p4FWvYabjqzbkLrtTOqi8U6e', status=None, thread_id='thread_pnWAMRydead3xtTeIpvHTYTs')], object='list', first_id='msg_hHncd621QmjDxbsQEiRAvQUw', last_id='msg_hHncd621QmjDxbsQEiRAvQUw', has_more=False)

In [24]:
ensure_vector_store(response.data[0])

<coroutine object ensure_vector_store at 0x70db327903c0>

## As in openai_assistant.py

In [41]:
# Helper function to create an assistant, if not found
@backoff.on_exception(backoff.expo, Exception, max_tries=5)
async def create_assistant():
    try:
        # Making a call to create an assistant
        assistant = client.beta.assistants.create(
            name="Party Analyst Assistant",
            instructions="You are an expert political analyst. Use your knowledge base to answer questions about party positions.",
            model="gpt-4-turbo",
            tools=[{"type": "file_search"}],
        )
        print(f"Assistant created with ID: {assistant.id}")
        return assistant
    except Exception as e:
        print("Failed to create assistant:", e)
        raise HTTPException(status_code=500, detail=f"Failed to create assistant: {str(e)}")


In [58]:
# Helper function to get an assistant, returns the ID of the Political Analyst Assistant if found and creates a new one if not found
@backoff.on_exception(backoff.expo, Exception, max_tries=5)
async def get_assistant():
    try:
        # Making a call to list assistantsassistant
        response = client.beta.assistants.list()  # Fetching all the assistants
        if response and response.data:
            for assistant in response.data:
                if assistant.name == "Party Analyst Assistant":
                    print("Party Analyst Assistant found.")
                    return assistant
            # If no assistant found in the loop, create a new one
            print("No Party Analyst Assistant found within, creating a new one...")
            assistant = await create_assistant()
            return assistant
        else:
            print("No assistants or data available or failed to fetch data, attempting to create a new assistant.")
            assistant = await create_assistant()
            return assistant
    except Exception as e:
        print("Failed to fetch assistants:", e)
        raise HTTPException(status_code=500, detail=f"Failed to fetch assistants: {str(e)}")


In [43]:
# Helper function to ensure the vector store is attached to the assistant
@backoff.on_exception(backoff.expo, Exception, max_tries=5)
async def ensure_vector_store(assistant):
    try:
        # Check if the current assistant has the correct vector store attached
        vector_store_ids = assistant.tool_resources.file_search.vector_store_ids if assistant.tool_resources and assistant.tool_resources.file_search else []

        if vector_store_ids:
            # Fetch the vector store details
            vector_store = client.beta.vector_stores.retrieve(vector_store_id=vector_store_ids[0])
            if vector_store and vector_store.name == "Party Positions":
                print("Correct vector store already attached.")
                return vector_store
            else:
                print("Incorrect vector store attached, looking for correct vector store.")
        else:
            print("No vector store attached, looking for correct vector store.")

        # If the correct vector store is not attached, check if such a store exists
        all_stores = client.beta.vector_stores.list()
        party_positions_store = next((store for store in all_stores.data if store.name == "Party Positions"), None)

        if party_positions_store:
            vector_store_id = party_positions_store.id
            print(f"Vector store 'Party Positions' found with ID: {vector_store_id}")
        else:
            # Create a new vector store if not found
            print("Creating new vector store 'Party Positions'")
            vector_store = client.beta.vector_stores.create(name="Party Positions")
            vector_store_id = vector_store.id

            # Upload files to the new vector store
            file_paths = ["../data/party_positions.pdf"] # File path to the PDF file
            file_streams = [open(path, "rb") for path in file_paths]
            file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
                vector_store_id=vector_store_id, files=file_streams
            )
            print(f"Files uploaded to vector store: {file_batch.status}")

        # Attach the vector store to the assistant
        updated_assistant = client.beta.assistants.update(
            assistant_id=assistant.id,
            tool_resources={"file_search": {"vector_store_ids": [vector_store_id]}}
        )
        print(f"Vector store '{vector_store_id}' attached to assistant.")
        return vector_store

    except Exception as e:
        print("Failed to ensure vector store:", e)
        raise HTTPException(status_code=500, detail=f"Failed to ensure vector store: {str(e)}")


In [62]:
# Helper Function to create a new conversation thread
@backoff.on_exception(backoff.expo, Exception, max_tries=5)
async def create_conversation(assistant, vector_store):
    try:
        # Create a new conversation thread
        thread = client.beta.threads.create(
                            messages=conversation_start,
                            tool_resources={
                                "file_search": {
                                    "vector_store_ids": [vector_store.id]
                                }
                            }
                        )
        print(f"Conversation thread created with ID: {thread.id}")
        
        # Create and poll the run
        run = client.beta.threads.runs.create_and_poll(
            thread_id=thread.id, assistant_id=assistant.id
        )
        print(f"Run created with ID: {run.id}")
        return thread, run
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Failed to create conversation: {str(e)}")

In [124]:
# Main function to run the setup
@backoff.on_exception(backoff.expo, Exception, max_tries=5)
async def assistant_setup():
    try:
        # Step 1: Get or create the assistant
        assistant = await get_assistant()

        # Step 2: Ensure the assistant has the correct vector store attached
        vector_store = await ensure_vector_store(assistant)
        
        # Step 3: Create a new conversation thread
        thread, run = await create_conversation(assistant, vector_store)
        
        # Step 4: Get the response from the assistant
        conversation = {}
        response = client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id)
        conversation['assistant'] = [response.data[0].content[0].text.value]
        conversation['user'] = []
        return assistant, vector_store, thread, run, conversation

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [46]:
system_prompt = """
You're an AI political guide designed to engage in Socratic dialogue. Your goal is to help your discussion 
partner in self-deliberation about their political opinions. Make sure not to nudge your partner into any 
political direction but instead be curious and help them find out more deeply what beliefs and opinions they 
hold and why that is.

Begin the conversation with a greeting and and invitation to the discussion partner to share the political 
topics that concern them the most. Prompt them to select one topic to delve into first.

Opening Inquiry: Start with an open-ended question to explore their initial thoughts about the chosen topic:
'What concerns you most about this topic and why do you think it's important?'

Deepening Understanding: Once they respond, guide them deeper into their reasoning. Ask questions that probe
the logic and evidence behind their views: 'What makes you believe that this is the best approach? Can you 
share examples or evidence that support your opinion?'

Introducing Challenges: After understanding their argument, introduce a counterpoint or challenge to their 
view to test the robustness of their reasoning: 'Have you considered [a specific contradiction or different 
perspective]? How does this aspect affect your viewpoint?'

Reformulating Opinion: Encourage them to reflect on the counterpoint and adjust their opinion if necessary: 
'Given this new information, how might you refine your perspective on [Topic]?'

Continuation or Change: Before moving on, ask if they want to delve deeper into the same topic or switch to 
another concern: 'Would you like to explore this topic further, or shall we discuss another one of your 
concerns?'

Repeat these steps for each topic they are concerned about. Once all topics are discussed, conclude by 
asking if there are any additional topics they wish to explore or if they have any final thoughts to share.

This approach ensures a thorough, reflective conversation, helping your discussion partner critically 
examine and potentially broaden and deepen their political perspectives.
"""


conversation_start = [{
        "role": "user",
        "content": system_prompt,
    }]

In [125]:
assistant, vector_store, thread, run, conversation = await assistant_setup() 
conversation


Party Analyst Assistant found.
Correct vector store already attached.
Conversation thread created with ID: thread_lNO64NtObYu0vSRsY3ZkwN7H
Run created with ID: run_PguHmmBOLDeoZNMtWzV41Sy9


In [138]:
# Main Chatbot Completion Function for assistants API
@backoff.on_exception(backoff.expo, Exception, max_tries=5)
async def chatbot_completion(
    message,
    assistant,
    thread,
    run,
    conversation
    ):
    try:
        conversation["user"].append(message)
        # Create a message to append to our thread
        message = client.beta.threads.messages.create(
            thread_id=thread.id, role='user', content=message)
        
        # Execute our run
        run = client.beta.threads.runs.create_and_poll(
            thread_id=thread.id,
            assistant_id=assistant.id,
        )
        response = client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id)
        conversation['assistant'].append(response.data[0].content[0].text.value)

        return conversation
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


In [140]:
conversation = await chatbot_completion(
    message="I believe a lot of people that belong here and are working hard are not getting the jobs they need to sustain their families.",
    assistant=assistant,
    thread=thread,
    run=run,
    conversation=conversation
    )
conversation

{'assistant': ["Hello! I'm here to help you explore and reflect on your political opinions through a Socratic dialogue. To begin, could you share some political topics that currently concern you the most? Please feel free to list a few, and then we can select one to delve into deeply first.",
  "Thank you for sharing your concern about job security and the impact of immigration on employment. Let's delve into this topic a bit deeper.\n\nWhat concerns you most about this topic and why do you think it's important?",
  "It sounds like you're deeply concerned about the welfare and financial stability of long-standing communities and their ability to find sustainable employment. \n\nWhat makes you believe that the presence of immigrants is a primary factor affecting job availability for locals? Can you share any specific examples or evidence that support your opinion?"],
 'user': ["It's the job security, the immigrants are taking our jobs!",
  "It's the job security, the immigrants are taki